## Set up a duckweed genotypes x media growth assay

### Pre-requisites to use this script
1. Precise plate and well positions for your machine defined in Plate_positions.py in the 'utils' subdirectory. 
2. All the parameters defined in the cell 'User paramters'. 
3. Sufficient 24-well plates, sterile media and duckweed plants available. 
4. Jubilee machine set up with Media-dispensing syringe tool (50 mL), Duckweed transfer syringe tool and lab automation bedplate. 


In [1]:
#Importing python libraries downloaded from the internet
import random
import pandas as pd
import os
import json
import time

from plantcv import plantcv as pcv

In [38]:
#Importing python libraries from local files. 
from utils.CameraUtils import *
import utils.DuckbotExptSetupUtils as exp
from utils.MachineUtils import *
import utils.PlatePositionUtils as pp 

In [3]:
port = "/dev/ttyACM0"
m = MachineCommunication(port)

## 1. Define user parameters

### Labware config 
All should be in arbitrary Jubilee motor units. Find this manually on the machine. They shouldn't need to be updated until you start using a new reservoir. And if that's the case you could create a labware library and then define the media_reservoir variable as one of the objects in your library. 

In [4]:
#X and Y should get the machine to the center of the reservoir. 
media_reservoir = { 'x' : 75, 'y' : 241,}

#Tool positions
media_syringe = 2 #What jubilee tool position did you define this as?
duckweed_syringe = 3
inoculation_loop = 4

#Volume calculations
dispenses_per_syringe_fill = 20
dispense_mL = 1.5 #In mL
vol_conversion = 3.9 #One mL is 3.9 units. 
dispense_offset = dispense_mL * vol_conversion

#Z-positions for different actions
z_dict = {"zero": 0, #Note that zeros
          "aspirate" : -46.5, #Measure this for yourself and your labware/toolheads.
          "dispense" : -21.5
         }

### Define experimental variables, and desired file names and save locations. 

In [15]:

#1. DEFINE EXPERIMENTAL VARIABLES
# genotypes = ["Sp7498", "Lm5500", "Lm8627"] # Replace with names for unique duckweed genotypes
genotypes = ["Sp7498", "Lm5500"] # Replace with names for unique duckweed genotypes
media = ["Mock"] # Replace with names for unique media

# media = ["Mock", "25 mM Salt", "50 mM Salt", "75 mM Salt","150 mM Salt"] # Replace with names for unique media
reps  = 12 # Replace with your desired number of replicates for each duckweed/media combination. 

#2. DEFINE FILE LOCATION AND NAME
expt_setup_parent_dir = os.getcwd() # Default uses current working directory but you can replace with your own choice. 
# expt_setup_dir = "TestDriveExpt_1_08032022" # Name of the folder to hold experiment data and metadata including the setup file
expt_setup_dir = "SandboxAspiration" 
# expt_setup_filename = "TestDriveExpt_1_08032022.json" #Name for the experiment setup file (Metadata)
expt_setup_filename = "SandboxAspiration.json" #Name for the experiment setup file (Metadata)


expt_setup_file_path = os.path.join(expt_setup_parent_dir, expt_setup_dir)
print(expt_setup_file_path)

if not os.path.exists(expt_setup_file_path):
    os.mkdir(expt_setup_file_path)     

/home/pi/duckbot/notebooks/SandboxAspiration/SandboxAspiration


## 2. Create dataframe with experiment metadata

In [16]:
# Creates master list of sample info, shuffles and then assigns to plates and wells. 
master_expt_list = []

for g in genotypes:
    for md in media:
        for x in range(reps):
             master_expt_list.append({"genotype": g, "media": md, "condition_replicate": x + 1})


random.shuffle(master_expt_list)
master_expt_list = exp.assign_plates_and_wells(master_expt_list)
expt_dict = {"sample_info" : master_expt_list}

In [17]:
#Save experimental set up file
os.chdir(expt_setup_file_path)
with open(expt_setup_filename, 'w') as f:
    json.dump(expt_dict, f)

In [18]:
#Import from file (in case user wants to make any manual edits to the JSON file after creating it)

with open(expt_setup_filename) as datafile:
    expt_data = json.load(datafile)

# Turn samples list into a dataframe
sample_data = expt_data["sample_info"]

## 3. Label 24-well plates and add to machine

#### Label Plates and add plates to machine

In [19]:
df = pd.DataFrame(sample_data)
num_plates = df.Plate.nunique()
print(num_plates)

print("This experiment requires {} 24-well plate(s)".format(num_plates))
print("----")
lst = list(range(1,num_plates + 1))
for n in lst:
    print("Label a plate with experiment ID or initials and 'plate {}'".format(n))
print("----")
print ("Place the 24-well plate(s) in the jubilee".format(num_plates))
print ("Start at position 1 and fill empty plate slots in order")

1
This experiment requires 1 24-well plate(s)
----
Label a plate with experiment ID or initials and 'plate 1'
----
Place the 24-well plate(s) in the jubilee
Start at position 1 and fill empty plate slots in order


## 2. Dispense media
When prompted insert containers of the relevant sterile media into the input slot on the Jubilee. 

In [20]:
#Retrieve absolute positions of wells from a library and then add those coordinatest to the plate set up dataframe
df = pp.add_well_coords_to_df_from_file(expt_setup_file_path, expt_setup_filename)

In [21]:
#Reorganizes dataframe to create machine instructions sorted by media-type
media_dicts = pp.pull_list_of_well_coord_dicts_by_dfcolumn(df, 'media')
print(media_dicts)

[{'media': 'Mock', 'well-coords': [[29.0, 175.0], [48.0, 175.0], [67.0, 175.0], [86.0, 175.0], [105.0, 175.0], [124.0, 175.0], [29.0, 156.0], [48.0, 156.0], [67.0, 156.0], [86.0, 156.0], [105.0, 156.0], [124.0, 156.0], [29.0, 137.0], [48.0, 137.0], [67.0, 137.0], [86.0, 137.0], [105.0, 137.0], [124.0, 137.0], [29.0, 118.0], [48.0, 118.0], [67.0, 118.0], [86.0, 118.0], [105.0, 118.0], [124.0, 118.0]]}]


In [22]:
#Pick up syringe toolhead
# port = "/dev/ttyACM0"
# m = MachineCommunication(port)
m.toolChange(media_syringe)

In [ ]:
# Send machine instructions
for media in media_dicts:
    m.moveTo(x=0,y=0,z=0)
    print(f"Please ensure {media['media']} is available in the machine before continuing.")
    print("Change syringe and/or needle if desired")
    while True:
        value = input("Enter 'YES' to confirm that the correct media is in position")
        if value != "YES":
            print("Please confirm")
        else:
            break
    exp.dispense_to_wells(m, media["well-coords"], dispense_offset, dispenses_per_syringe_fill, media_reservoir, z_dict)

Please ensure Mock is available in the machine before continuing.
Change syringe and/or needle if desired
Enter 'YES' to confirm that the correct media is in positionYES
Move to Z = zero
Move to reservoir position
moved to height for aspiration
Moved to Z = zero
Hovering over the first well to dispense into
Prepare to dispense
X = 29.0
Y = 175.0
Prepare to dispense
X = 48.0
Y = 175.0
Prepare to dispense
X = 67.0
Y = 175.0
Prepare to dispense
X = 86.0
Y = 175.0
Prepare to dispense
X = 105.0
Y = 175.0
Prepare to dispense
X = 124.0
Y = 175.0
Prepare to dispense
X = 29.0
Y = 156.0
Prepare to dispense
X = 48.0
Y = 156.0
Prepare to dispense
X = 67.0
Y = 156.0
Prepare to dispense
X = 86.0
Y = 156.0
Prepare to dispense
X = 105.0
Y = 156.0
Prepare to dispense
X = 124.0
Y = 156.0
Prepare to dispense
X = 29.0
Y = 137.0
Prepare to dispense
X = 48.0
Y = 137.0
Prepare to dispense
X = 67.0
Y = 137.0
Prepare to dispense
X = 86.0
Y = 137.0
Prepare to dispense
X = 105.0
Y = 137.0
Prepare to dispense
X =

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/pi/duckbot/.venv/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_2405/4087302342.py", line 12, in <module>
    exp.dispense_to_wells(m, media["well-coords"], dispense_offset, dispenses_per_syringe_fill, media_reservoir, z_dict)
  File "/home/pi/duckbot/notebooks/utils/DuckbotExptSetupUtils.py", line 55, in dispense_to_wells
    m.move(de=-dispense_offset, s=1000)
  File "/home/pi/duckbot/notebooks/utils/MachineUtils.py", line 94, in move
    self.send(cmd)
  File "/home/pi/duckbot/notebooks/utils/MachineUtils.py", line 22, in send
    self.ser.write(bcmd)
  File "/home/pi/duckbot/.venv/lib/python3.7/site-packages/serial/serialposix.py", line 640, in write
    abort, ready, _ = select.select([self.pipe_abort_write_r], [self.fd], [], None)
KeyboardInterrupt

During handling of the above exception, another exception occurred:



ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/pi/duckbot/.venv/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_2405/4087302342.py", line 12, in <module>
    exp.dispense_to_wells(m, media["well-coords"], dispense_offset, dispenses_per_syringe_fill, media_reservoir, z_dict)
  File "/home/pi/duckbot/notebooks/utils/DuckbotExptSetupUtils.py", line 55, in dispense_to_wells
    m.move(de=-dispense_offset, s=1000)
  File "/home/pi/duckbot/notebooks/utils/MachineUtils.py", line 94, in move
    self.send(cmd)
  File "/home/pi/duckbot/notebooks/utils/MachineUtils.py", line 22, in send
    self.ser.write(bcmd)
  File "/home/pi/duckbot/.venv/lib/python3.7/site-packages/serial/serialposix.py", line 640, in write
    abort, ready, _ = select.select([self.pipe_abort_write_r], [self.fd], [], None)
KeyboardInterrupt

During handling of the above exception, another exception occurred:



## 5. Transfer duckweed
Place a container filled with fronds of the relevant duckweed type and the machine will attempt to move individual fronds into the relevant wells. After each attempt at filling all wells a camera will take pictures of each well to confirm success and then unsuccessful wells will be reattempted

In [23]:
duckweed_reservoir = [75, 241]
#pop the bed down to avoid any collisions on tool change
m.moveTo(z=50)

In [24]:
# pick up the innoculation loop
m.toolChange(inoculation_loop)
m.moveTo(x=duckweed_reservoir[0], y=duckweed_reservoir[1])

In [35]:
# find suitable z heights to collect, move, and drop-off duckweed
# this can be done using the duet web control interface
inoculation_loop_z_dict = {"collect": -34.7,  # what height fully immerses the inoculation loop in the reservoir?
                          "move" : -7, # what height clears all labware to move between reservoir/well plates?
                          "dispense" : -32  # what height fully immerses the innoculation loop in the well plate?
         }
# collect_height = -34.7 #
# move_height = -7 
# transfer_height = -32

In [36]:
# pop the bed down to avoid collisions after any probing 
m.moveTo(z=50)

In [39]:
# Inoculation Loop Transfer
# The machine will move after running this cell
exp.inoculation_loop_transfer(m, df, inoculation_loop_z_dict)
# grouped_df = df.groupby('genotype')
# for field_value, sample_df in grouped_df:
#     print("Place container of duckweed type **{0}** into jubilee and ensure lid is open".format(field_value))
#     print("""Type anything into the input field to confirm that the media is available.
#     After this point the Jubilee will begin dispensing""")
#     input() 
#     count = 0
#     for index,s in sample_df.iterrows():
#         #move to plate and well
#         #move to plate and well
#         r = 20
#         rx = random.randint(-r, r)
#         ry = random.randint(-r, r)

#         # move in xy first
#         m.moveTo(x=duckweed_reservoir[0] + rx, y=duckweed_reservoir[1] + ry)

#         # dip into the reservoir
#         m.moveTo(z=collect_height)
#         # slowly sweep
#         m.move(dx=5, s=100)
#         m.move(dy=5, s=100)
# #         m.dwell(1000) # wait a bit
#         m.move(dz=10, s=250) # start moving slowly up
#         m.moveTo(z=move_height)
#         well = pp.fetch_well_position(s["Plate"][-1], str(s["Well"]))
#         m.moveTo(x=well['x'], y=well['y'])
#         m.moveTo(z=transfer_height)
#         m.move(dx=3, s=100)
#         m.move(dy=-3, s=100) # move in opposite direction
#         m.dwell(250)
#         m.moveTo(z=move_height, s=800)

AttributeError: module 'utils.DuckbotExptSetupUtils' has no attribute 'inoculation_loop_transfer'

In [28]:
# pop bed down to access labware
m.moveTo(z=100)

## Print out instructions for manual transfer of fronds

In [ ]:
#Manual transfer

grouped_df = df.groupby('genotype')
for field_value, sample_df in grouped_df:
    print("Place container of duckweed type **{0}** into jubilee and ensure lid is open".format(field_value))
    for index,s in sample_df.iterrows():
        plate = s["Plate"]
        well = s["Well"]
        print(f"Transfer {field_value} to plate {plate}, well {well}")
    input()
        #print("Dispensing media of type {0} into {1}, well {2}".format(field_value,s["Plate"], s["Well"]))


In [ ]:
# print("Write down any notes about today's set up that you would like to be recorded in the set up file")
# notes = input()
# #Save experimental set up file
# # os.chdir(expt_setup_file_path)

# with open(expt_setup_filename) as datafile:
#     expt_data = json.load(datafile)
#     expt_data["Set_up_notes"] = notes
#     #Save file?
#     print(expt_data)

## Consecutive Transfer Passes

In [29]:
#Looks for the outline of the well that the duckweed are in and crops to that outline. 
def circle_crop(img):
    img1 = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    blur = cv2.medianBlur(gray, 5)
    ret, thresh = cv2.threshold(gray, 100, 150, cv2.THRESH_BINARY)
    # Create mask
    height, width, nslice = img.shape
    mask = np.zeros((height,width), np.uint8)
    cimg=cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    edges = cv2.Canny(gray, 25, 100)
    gain = 1
    for g in np.arange(gain, 10, 0.1):
        # find the right gain to find a single circle
        circles = cv2.HoughCircles(edges,
                                   cv2.HOUGH_GRADIENT,
                                   g,
                                   minDist=100,
                                   param1=100,
                                   param2=100,
                                   minRadius=325,
                                   maxRadius=650
                                   )
        if circles is None:
            continue
        numCircles = circles[0,:].shape[0]
        if numCircles == 1:
            break

#     circles = cv2.HoughCircles(edges, cv2.HOUGH_GRADIENT, 1, 100, param1 = 100, param2 = 30, minRadius = 325, maxRadius = 650)
    if circles is not None:
        i = circles[0][0]                
        # Draw on mask
        center = (int(i[0]),int(i[1]))
        radius = int(i[2])
        cv2.circle(mask,center,radius,(255,255,255),thickness=-1)
# Copy that image using that mask
        masked_data = cv2.bitwise_and(img1, img1, mask=mask)
        # Apply Threshold
        _,thresh = cv2.threshold(mask,1,255,cv2.THRESH_BINARY)
        # Find Contour
        contours = cv2.findContours(thresh,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
        x,y,w,h = cv2.boundingRect(contours[0][0])
    #   Crop masked_data
        cropped_img = masked_data[y:y+h,x:x+w]
    else:
        cropped_img = "Error"
    return(np.ascontiguousarray(cropped_img))

def identify_fronds(img):
    s = pcv.rgb2gray_hsv(img, 's')
    s_thresh = pcv.threshold.binary(s, 120, 255, 'light')
    objects, hierarchy = cv2.findContours(s_thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)[-2:]
    # Cast tuple objects as a list
    objects = list(objects)
    count = 0
    for i, cnt in enumerate(objects):
        area = cv2.contourArea(cnt)
        perimeter = cv2.arcLength(cnt,True)
        # if area > 1000 and area < 2000: # and perimeter > 100:
        if perimeter > 100 and area > 500: # can add stricter filters here as necessary
#             print(f"perimeter: {perimeter}\n area: {area}")
            count += 1
#             M = cv2.moments(cnt)
#             cx = int(M['m10']/M['m00'])
#             cy = int(M['m01']/M['m00'])
#             cv2.circle(crop, (cx, cy), 10, (255,0,0), -1)
#             cv2.drawContours(crop, objects, i, (255, 102, 255), 2, lineType=8, hierarchy=hierarchy)
    
    return count
    

In [30]:
def check_wells(df):
    has_fronds = []
    for index, row in df.iterrows():
        plate = row['Plate']
        well = row['Well']
        well_x = row['x'] 
        well_y = row['y']
        print(f'Checking well {well}')
        m.moveTo(x=well_x, y=well_y, z=10)
        m.dwell(500) #dwell .75 seconds
        f = getFrame()
        crop = circle_crop(f)
        frond_check = identify_fronds(crop)
        if frond_check:
#             print(f"Well {well} has fronds!")
            has_fronds.append(True)
        else:
            print(f"Well {well} needs a frond")
            has_fronds.append(False)
        time.sleep(0.1)
    # update the df with the a new column
    df['hasFronds'] = has_fronds
    
def fill_empty_wells(df):
    m.moveTo(z=50)
    m.toolChange(4)
    empty = df.loc[df['hasFronds'] == False]
    grouped_df = empty.groupby('genotype')
    for field_value, sample_df in grouped_df:
        print("Place container of duckweed type **{0}** into jubilee and ensure lid is open".format(field_value))
        print("""Type anything into the input field to confirm that the media is available.
        After this point the Jubilee will begin dispensing""")
        input() 
        count = 0
        for index,s in sample_df.iterrows():
            #move to plate and well
            #move to plate and well
            r = 20
            rx = random.randint(-r, r)
            ry = random.randint(-r, r)

            # move in xy first
            m.moveTo(x=duckweed_reservoir[0] + rx, y=duckweed_reservoir[1] + ry)

            # dip into the reservoir
            m.moveTo(z=collect_height)
            # slowly sweep
            m.move(dx=5, s=100)
            m.move(dy=5, s=100)
    #         m.dwell(1000) # wait a bit
            m.move(dz=10, s=250) # start moving slowly up
            m.moveTo(z=move_height)
            well = pp.fetch_well_position(s["Plate"][-1], str(s["Well"]))
            m.moveTo(x=well['x'], y=well['y'])
            m.moveTo(z=transfer_height)
            m.move(dx=3, s=100)
            m.move(dy=-3, s=100) # move in opposite direction
            m.dwell(250)
            m.moveTo(z=move_height, s=800)
        
    
            


In [31]:
port = '/dev/ttyACM0'
m = MachineCommunication(port)
m.toolChange(1)

In [32]:
check_wells(df)

Checking well A1
Checking well A2
Checking well A3
Checking well A4
Checking well A5
Checking well A6
Checking well B1
Checking well B2
Checking well B3
Checking well B4
Checking well B5
Checking well B6
Checking well C1
Checking well C2
Checking well C3
Checking well C4
Checking well C5
Checking well C6
Checking well D1
Checking well D2
Checking well D3
Checking well D4
Checking well D5
Checking well D6


In [33]:
fill_empty_wells(df)

Place container of duckweed type **Lm5500** into jubilee and ensure lid is open
Type anything into the input field to confirm that the media is available.
        After this point the Jubilee will begin dispensing
y
Place container of duckweed type **Sp7498** into jubilee and ensure lid is open
Type anything into the input field to confirm that the media is available.
        After this point the Jubilee will begin dispensing
y


## Syringe Aspiration

In [ ]:
port = '/dev/ttyACM0'
m = MachineCommunication(port)
m.toolChange(3)

In [ ]:
# manually probe the z height of the duckweed reservoir surface
# enter the z value here
# port = '/dev/ttyACM0'
# m = MachineCommunication(port)
duckweed_reservoir = [77, 243, -17.3]
m.moveTo(x=duckweed_reservoir[0], y=duckweed_reservoir[1])
# center (x=77): -17.1
# edge (x=109): -17.5

In [ ]:
# make sure z value is >0 again after probing water height
m.moveTo(z=80) 

In [ ]:
# grab the camera, move the relevant height & position to take pic of duckweed
m.toolChange(1)
m.moveTo(x=duckweed_reservoir[0], y=duckweed_reservoir[1])

# move down more based on surface height
# syringe_zero = 22.9 # for blue syringe
# syringe_zero = 24.6 # for black syringe tip
syringe_zero = 23.3 # for pink syringe tip
dh = syringe_zero + duckweed_reservoir[2]
m.move(dz=dh)

In [ ]:
# load in the relevant calibration files

# with open("/home/pi/autofocus-test/JubileeAutofocus/camera_cal_z_60_tape.json") as f:
with open("/home/pi/autofocus-test/JubileeAutofocus/camera_cal_z_80.json") as f:
    cal = json.load(f)
matrix = np.array(cal['transform'])
size = cal['resolution']
print(matrix)
print(size)

m.transform = matrix
m.img_size = size

In [ ]:
#focus camera
cap = cv2.VideoCapture(0) #Note that the index corresponding to your camera may not be zero but this is the most common default

# draw a circle in the center of the frame
center = None
while center is None:
    # the first frame grab is sometimes empty
    ret, frame = cap.read()
    h, w = frame.shape[0:2]
    center = (int(w/2), int(h/2))

while True:
    ret, frame = cap.read()
    target = cv2.circle(frame, center, 5, (0,255,0), -1)
    cv2.imshow('Input', frame)
    c = cv2.waitKey(1)
    if c ==27: #27 is the built in code for ESC so press escape to close the window. 
        break 
        
cap.release()
cv2.destroyAllWindows()

In [ ]:
# manually pick fronds
# alternatively, manually click a frond
f = getFrame()
# saveFrame(f, "/home/pi/Downloads/petri-60.jpg")
pts = selectPoint(f, num_pts=8)

In [ ]:
# convert px coord to real coord
def px_to_real(x,y, absolute = False):
        x = (x / m.img_size[0]) - 0.5
        y = (y / m.img_size[1]) - 0.5
        a = 1 if absolute else 0

        return (m.transform.T @ np.array([x**2, y**2, x * y, x, y, a]))

fronds = []
for pt in pts:
    frond_off = px_to_real(pt[0], pt[1])
    frond = [duckweed_reservoir[0] - frond_off[0], duckweed_reservoir[1] - frond_off[1]]
    fronds.append(frond)

In [ ]:
# switch tools, account for offset
m.toolChange(3)
syringe_off = [-0.76, 4.2] # pink
# syringe_off = [-1, 5] # green
# syringe_off = [-1.1, 5] # black
# syringe_off = [0.2, 4.86] # new blue
# syringe_off = [-0.68, 4.82] # old blue
# 73.8, 227.9
print(fronds)

In [ ]:
# test block to confirm camera-to-machine alignment
idx = 0
fx = fronds[idx][0]
fy = fronds[idx][1]
fx, fy

m.moveTo(x=fx, y=fy)
showFrame(getFrame())

In [ ]:
# testing camera to syringe tool alignment
m.moveTo(m.moveTo(x=fx + syringe_off[0], y=fy + syringe_off[1]))

In [ ]:
# aspirate duckweed
# first move to surface
def aspirate():
#     m.move(de=5, s=1000) ; # 'prime' syringe
    m.moveTo(z=duckweed_reservoir[2])
    m.dwell(1000)
    m.move(dz=-3) # press slightly
    m.dwell(1000)
    m.move(de=20, s=1000) # aspirate!
    m.move(dz=6, s=500) # aspirate!
#     m.move(dz=5, de=50, s=1000) # aspirate!

def aspirate_2():
#     m.move(de=5, s=1000) ; # 'prime' syringe
    m.moveTo(z=duckweed_reservoir[2] + 2)
    m.dwell(1000)
    m.move(dz=-3.5, de=5) # press slightly
#     m.dwell(250)
    m.move(de=40, s=1800) # aspirate! #1200 for lemna minor
    m.move(dz=6, s=500, de=5) # aspirate!
#     m.move(dz=5, de=50, s=1000) # aspirate!

def dispense():
    m.dwell(1000)
    m.move(de=-50, s=1400) # dispense
#     m.move(de=-40, s=1800)
    m.dwell(500)

grouped_df = df.groupby('genotype')
for field_value, sample_df in grouped_df:
    print("Place container of duckweed type **{0}** into jubilee and ensure lid is open".format(field_value))
    print("""Type anything into the input field to confirm that the media is available.
    After this point the Jubilee will begin dispensing""")
    input() 
    count = 0
    for index,s in sample_df.iterrows():
        #move to plate and well
        frond = fronds[count]
        m.moveTo(m.moveTo(x=frond[0] + syringe_off[0], y=frond[1] + syringe_off[1]))
#         aspirate()
        aspirate_2()
        m.moveTo(z=6.5)
        well = pp.fetch_well_position(s["Plate"][-1], str(s["Well"]))
        m.moveTo(x=well['x'], y=well['y'])
        m.dwell(1000)
        dispense()
        count += 1
        if count > 7:
            break
# for frond in fronds:
#     m.moveTo(m.moveTo(x=frond[0] + syringe_off[0], y=frond[1] + syringe_off[1]))
#     aspirate()
#     m.dwell(3000)
#     dispense()
    